# Notebook for creation of models

In [1]:
import pickle
import os
import pandas as pd
os.chdir('/home/carlotaalv/Escritorio/TFM/tfm/')
import models as mod
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import re
from tsfresh.feature_extraction import extract_features, EfficientFCParameters,MinimalFCParameters


In [2]:
os.chdir('data_models')

### Carga de dataframes finales

In [3]:
X_data_all=pd.read_csv('all_data_unproc_index.csv')
Y_data_all=pd.read_csv('Y_values_unproc_index.csv')
Y_data_barnase_alphalactalbumin=pd.read_csv('data_Y_barnase_alphalact.csv')
X_data_barnase_alphalactalbumin=pd.read_csv('data_X_barnase_alphalact.csv')


In [4]:
X_data_all_complete= pd.concat([X_data_all, X_data_barnase_alphalactalbumin], axis=0, ignore_index=True)
Y_data_all_complete=pd.concat([Y_data_all, Y_data_barnase_alphalactalbumin], axis=0, ignore_index=True)

In [5]:
X_data_all_complete.to_csv('all_data_unproc_index_complete.csv',index=False)
Y_data_all_complete.to_csv('Y_values_unproc_index_complete.csv',index=False)


In [6]:
X_halm2=X_data_all_complete[X_data_all_complete['prot']=='halm2_AF_apo']
Y_halm2=Y_data_all_complete[Y_data_all_complete['prot']=='halm2_AF_apo']
X_data = X_data_all_complete[X_data_all_complete['prot'] != 'halm2_AF_apo']
Y_data = Y_data_all_complete[Y_data_all_complete['prot'] != 'halm2_AF_apo']

In [7]:
X_halm2.to_csv('X_halm2_data.csv',index=False)
Y_halm2.to_csv('Y_halm2_data.csv',index=False)
X_data.to_csv('X_training_data_complete.csv',index=False)
Y_data.to_csv('Y_training_data_complete.csv',index=False)

In [8]:
X_halm2 = pd.read_csv('X_halm2_data.csv')
Y_halm2 = pd.read_csv('Y_halm2_data.csv')
X_data = pd.read_csv('X_training_data_complete.csv')
Y_data = pd.read_csv('Y_training_data_complete.csv')

### Features extraction with tsfresh

In [9]:
X_halm2_features,Y_halm2_features,Y_halm2_id=mod.x_y_extraction_models(X_halm2,Y_halm2,water=True)

Feature Extraction: 100%|██████████| 40/40 [00:01<00:00, 23.03it/s]


In [10]:
X_halm2_features.to_csv('halm2_X_vals.csv',index=False)
Y_halm2_features.to_csv('halm2_Y_vals.csv',index=False)
Y_halm2_id.to_csv('halm2_Y_ID_vals.csv',index=False)

In [11]:
X_training,Y_training,Y_training_id=mod.x_y_extraction_models(X_data,Y_data,water=True)

Feature Extraction: 100%|██████████| 40/40 [00:07<00:00,  5.23it/s]


In [15]:
X_training.to_csv('training_X_vals.csv',index=False)
Y_training.to_csv('training_Y_vals.csv',index=False)
Y_training_id.to_csv('training_Y_ID_vals.csv',index=False)

### Model training

In [16]:
model_list=[
    ['knn','No','knn'],
    ['knn','ANOVA','knn_anova'],
    ['DecisionTree','No','DecisionTree'],
    ['GradientBoosting','No','GradientBoosting'],
    ['RandomForest','No','RandomForest']
]
models_results=mod.models(X_training,Y_training,models_list=model_list,prot='basic_params_WATER_complete')

In [21]:
with open('models_final_complete.pkl','wb') as file:
    pickle.dump(models_results,file)

In [22]:
columns=['Modelo','Error medio','Desviación estándar']
# Create an empty DataFrame with these columns
data_models = pd.DataFrame(columns=columns)

In [23]:
# Evaluate Dummy Model
avg_dummy_error, sd_dummy_error, regressor_dummy = mod.evaluate_dummy_regressor(X_training, Y_training, n_repeats=10, strat='mean')

null_models=mod.models_null(Y_training_id)
# Prepare data models
models_info = [
    ('Dummy', avg_dummy_error, sd_dummy_error),
    ('Molecular weight', null_models[0][3], null_models[0][4]),
    ('Hydrophobicity', null_models[1][3], null_models[1][4]),
    ('Knn', models_results[0][4], models_results[0][5]),
    ('Knn con anova', models_results[1][4], models_results[1][5]),
    ('Decision Tree', models_results[2][4], models_results[2][5]),
    ('Gradient Boosting', models_results[3][4], models_results[3][5]),
    ('RandomForest', models_results[4][4], models_results[4][5])
]

# Add models to data_models
for model_name, error_mean, error_sd in models_info:
    new_row = pd.DataFrame([{
        'Modelo': model_name,
        'Error medio': round(error_mean, 3),
        'Desviación estándar': round(error_sd, 3)
    }])
    data_models = pd.concat([data_models, new_row], ignore_index=True)

data_models = pd.concat([data_models, new_row], ignore_index=True)
data_models.to_csv('results_models_complete.csv',index=False)


/tmp/ipykernel_464301/2484967122.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_models = pd.concat([data_models, new_row], ignore_index=True)


In [24]:
data_models

,Modelo,Error medio,Desviación estándar
0,Dummy,4.440,0.202
1,Molecular weight,4.669,0.252
2,Hydrophobicity,4.658,0.246
3,Knn,1.857,0.136
4,Knn con anova,2.102,0.185
5,Decision Tree,2.031,0.184
6,Gradient Boosting,1.270,0.104
7,RandomForest,1.059,0.103
8,RandomForest,1.059,0.103


In [20]:
os.getcwd()

'/home/carlotaalv/Escritorio/TFM/tfm/data_models'